# Homework 5 - Question 2

In [ ]:
import pycountry
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
import nltk
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filename = os.path.join('hillary-clinton-emails','Emails.csv')
df_email_initial = pd.read_csv(filename)
df_email_initial.head(2)

In [ ]:
df_email = df_email_initial
# df_email = df_email_initial[["ExtractedSubject", "ExtractedBodyText"]]
df_email = df_email.dropna(axis=0, how='any', subset=['ExtractedBodyText'])
df_email.head(2)

In [ ]:
countriesName = list(x.name for x in list(pycountry.countries))
results = pd.DataFrame(countriesName, columns=['country'])
results['sentiment'] = ""
results['frequency'] = ""
results.head(5)

In [ ]:
bad_countries_abbrev = ["RE", "FM", "TV", "AL", "AQ", "LA", "BEN", "and", "is", 'my', 'no', 'to', 'are', 'in', 'so', 'at',
                       'as', 'be', 'by', 'can', 'it', 'am', 'as', 'do', 'us', 'me', 'pm', 'ago', 'arm', 'mn', 'nor', 'lie']
bad_countries_abbrev = [abb.lower() for abb in bad_countries_abbrev]

In [ ]:
def formatCountry(word):
    return (' ' + word + ' ').lower()

def makeCountryTable(country):
    countryArr = [country.name.lower()]
    if(country.alpha_2.lower() not in bad_countries_abbrev):
        countryArr.append(formatCountry(country.alpha_2))
    if(country.alpha_3.lower() not in bad_countries_abbrev):
        countryArr.append(formatCountry(country.alpha_3))
    if hasattr(country, 'official_name'):
        countryArr.append(country.official_name.lower())
    if hasattr(country, 'common_name'):
        countryArr.append(country.common_name.lower())
    ## Some countries have the form : 'Iran, Islamic Republic of' => we want to 
    ## to transform it to : 'Iran' and 'Islamic Republic of Iran'
    if "," in country.name:
        split_list = country.name.split(',')
        countryArr.append((split_list[1] + ' ' + split_list[0]).lower())
        ## There are two entities named Virgin Islands, so we don't take the name alone.
        if split_list[0] !=  'Virgin Islands':
            countryArr.append((split_list[0]).lower())
    ## Special case: We add the name "Syria" that is common
    if country.name == 'Syrian Arab Republic':
        countryArr.append('syria')
    
    return countryArr    

In [ ]:
countryNamesAbbrev = list(makeCountryTable(x) for x in list(pycountry.countries))

In [ ]:
# attr_x = []
# for x in list(pycountry.countries):
#     if "," in x.name:
#         print(x.name)
#         ll = x.name.split(',')
#         print(ll[1] + ' ' + ll[0])
#         if ll[0] !=  'Virgin Islands':
#             print(ll[0])
# #         print(x)
# #         ADD IRAN TO LIST!!!!!! AND CHECK FOR OTHER COUNTRIES
# # attr_x
# for x in list(pycountry.countries):
#     if "Syria" in x.name:
#         print(x)

Sentiment score computations: Vader and liu hu lexicon

In [ ]:
# nltk.download('opinion_lexicon')
# nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank

In [ ]:
def vader_sentiment_computation(text):
    """
    Function from https://github.com/nltk/nltk/blob/develop/nltk/sentiment/util.py
    Output polarity scores for a text using Vader approach.
    :param text: a text whose polarity has to be evaluated.
    """
    vader_analyzer = SentimentIntensityAnalyzer()
    return vader_analyzer.polarity_scores(text).get('compound')

In [ ]:
def computation_liu_hu_lexicon(text):
    """
    Function inspiration from https://github.com/nltk/nltk/blob/develop/nltk/sentiment/util.py
    Helper function to compute the number of positive word, of negative words and of neutrals words.
    Output: (Number of positive words, number of negative words, number of neutral words)
    :param text: a text whose polarity has to be classified.
    """

    tokenizer = treebank.TreebankWordTokenizer()
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(text)]
#     print(tokenized_sent)
    pos_words = 0
    neg_words = 0
    neut_words = 0
    
    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
#             print('positive: ',word)
            pos_words += 1
        elif word in opinion_lexicon.negative():
#             print('negative: ', word)
            neg_words += 1
        else:
            neut_words += 1

    print('Positive words: ', pos_words)
    print('Negative words: ', neg_words)
    print('Neutral words: ', neut_words)
    return (pos_words, neg_words, neut_words)

In [ ]:
def liu_hu_lexicon_absolute(pos_words, neg_words):
    """
    Sentiment classification using Liu and Hu opinion lexicon.
    This function compute the liu hu scores by taking the majority voting.
    If the majority of the words are positive, the score will be 1, if it
    is negative, the score will be -1, otherwise the score will be 0.
    :param pos_words: The number of positive words in a text
    :param neg_words: The number of negative words in a text
    """

    if pos_words > neg_words:
        return 1
    elif pos_words < neg_words:
        return -1
    elif pos_words == neg_words:
        return 0

In [ ]:
def liu_hu_lexicon_relative(pos_words, neg_words):
    """
    Sentiment classification using Liu and Hu opinion lexicon.
    This function compute the liu hu scores by taking the number of 
    positive words over the total number of positive and negative words. Therefore it will
    ponderate the score with the total number of negative and positive words
    and output a continuous score between -1 and 1. -1 beeing when
    there is only negative words and 1 when there is only positive words.
    :param pos_words: The number of positive words in a text
    :param neg_words: The number of negative words in a text
    """

    if pos_words == 0 and neg_words == 0:
        return 0
    else:
        return pos_words / (pos_words + neg_words) * 2 - 1

In [ ]:
def liu_hu_lexicon_neutral_ponderated(pos_words, neg_words, neut_words):
    """
    Sentiment classification using Liu and Hu opinion lexicon.
    This function compute the liu hu scores by ponderating the number of positive
    words with the total number of positive and negative words. It will then ponderate this 
    score with the total number of neutral words.
    :param pos_words: The number of positive words in a text
    :param neg_words: The number of negative words in a text
    :param neut_words: The number of neutral words in a text
    """

    if pos_words == 0 and neg_words == 0:
        return 0
    else:
        return (pos_words / ( neut_words + pos_words + neg_words))

In [ ]:
def execute_liu_hu_lexicon_tests(text):
    """
    Input: A text on which to execute the lexicon task
    Output: The results of the three lexicon functions
    """
    (pos_words, neg_words, neut_words) = computation_liu_hu_lexicon(text)
    absolute = liu_hu_lexicon_absolute(pos_words, neg_words)
    relative = liu_hu_lexicon_relative(pos_words, neg_words)
    ponderated = liu_hu_lexicon_neutral_ponderated(pos_words, neg_words, neut_words)
    return (absolute, relative, ponderated)

Function to analyze sentiment over the data:

In [ ]:
## Initialize dictionnaries to store sentiment
vader_sentiment_dict = {}
liu_hu_absolute_dict = {}
liu_hu_relative_dict = {}
liu_hu_ponderated_dict = {}

## Initialize a counter to know which country words are used to
## find countries, to check for mistakes.
country_words_used = {}

In [ ]:
## Update a dictionary with the country as key and the result as value.
def update_sentiment_dict(country_name, dict_to_update, result):
    if(not dict_to_update.get(country_name)):
        dict_to_update[country_name] = [result]
    else:
        dict_to_update[country_name].append(result)

## Update the fictionary country_words_used
def updateCountryWordUsed(abrev):
    if(not country_words_used.get(abrev)):
        country_words_used[abrev] = 1
    else:
        country_words_used[abrev] += 1

In [ ]:
def analyzeSentimentInDf(row):
    emailContent = str(row.ExtractedBodyText).lower()
    foundCountryList = []
    
    ## Iteration over all countries
    for country in countryNamesAbbrev:
        foundCountry = False
        
        ## Iteration over all names for a specific country
        for abrev in country:
            if not foundCountry and abrev in emailContent:
                if len(foundCountryList) == 0:
                    print('------------------------------- EMAIL N0: ', row.Id, ' -------------------------------')
                    print('--------- COUNTRIES ---------')
                foundCountry = True
                foundCountryList.append(country[0])
                updateCountryWordUsed(abrev)
                print(country[0], ': ',abrev)
    
    ## If we found at least one country name in email, we compute sentiment scores
    if len(foundCountryList) != 0:
        print('-----------------------------------')
        vader_score = vader_sentiment_computation(emailContent)
        (abs_lex_score, rel_lex_score, pond_lex_score) = execute_liu_hu_lexicon_tests(emailContent)
        
        ## Update dictionnaries containing scores:
        for f_country in foundCountryList:
            update_sentiment_dict(f_country, vader_sentiment_dict, vader_score)
            update_sentiment_dict(f_country, liu_hu_absolute_dict, abs_lex_score)
#             if rel_lex_score != -1:
            update_sentiment_dict(f_country, liu_hu_relative_dict, rel_lex_score)
#             if pond_lex_score != -1:
            update_sentiment_dict(f_country, liu_hu_ponderated_dict, pond_lex_score)  
                
        
        print('--------- SENTIMENT SCORE ---------')
        print('Vader score: ', vader_score)
        print('Absolute Lexicon score: ', abs_lex_score)
        print('Relative Lexicon score: ', rel_lex_score)
        print('Ponderated Lexicon score: ', pond_lex_score)

        print('--------- EMAIL CONTENT ---------')
        print(emailContent)

In [ ]:
# sample_email = df_email.head(100)
sample_email = df_email
sample_email.apply(analyzeSentimentInDf, axis=1)
print('finished!')

In [ ]:
import operator
keywords_used = sorted(country_words_used.items(), key=operator.itemgetter(1), reverse=True)
keywords_used

## Aggregated sentiment tests

In [ ]:
def aggregateSentimentInformation(sentiment_dict):
    '''
    It will transform the sentiment list in a dictionary and return the same dict
    with the mean value for the input list of sentiment value instead of the list.
    Input: Dictionary with (country -> List of sentiment values)
    Output: Dictionary with (country -> a unique mean value)
    '''
    resultDict = {}
    for country in sentiment_dict:
        resultDict[country] = np.mean(sentiment_dict[country])
    return resultDict

In [ ]:
def makeAdvancedDfFromDict(dict_name, sentiment_dict):
    '''
    Take a dictionnary with (country -> List of sentiment values), ie. {'switzerland': [0.5, 0.1, 1]}
    And transform it to a dataframe with as a columm the [country, sentiment, type]
    where type is the dict_name. It will make one row per value in the list of sentiment value.
    Input: Dictionary with (country -> List of sentiment values)
    Output: Dataframe with columns  [country, sentiment, type]
    '''
    plot_row_lists = []
    for country in sentiment_dict:
        for val in sentiment_dict[country]:
            plot_row_lists.append({'country': country, 'sentiment': val, 'type': dict_name})
    return pd.DataFrame(plot_row_lists)

In [ ]:
def makeAdvancedPlotForAllSentimentDict(sentiment_dict_map, y_min= -1.1, y_max = 1.1):
    '''
    It will take a dictionary of sentiment dictionaries of the form:
    (name of the sentiment analysis technique -> sentiment dictionary), where
    sentiment dictionary := (country -> List of sentiment values)
    Output: Plot with all the dictionaries put together
    '''
    df_list = []
    for dict_name in sentiment_dict_map:
        df_list.append(makeAdvancedDfFromDict(dict_name, sentiment_dict_map[dict_name]))
    
    plot_df = pd.concat(df_list)
    
    ## Order of columns in plot
    order = pd.Series(aggregateSentimentInformation(sentiment_dict_map['vader'])).sort_values().index
    
    plt.ylim(y_min, y_max)
#     plot = sns.stripplot(x="country", y="sentiment", data = plot_df, order = order);
    plot = sns.barplot(x="country", y="sentiment", hue="type", data = plot_df, order=order, errwidth=0);
    plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
    return plot

In [ ]:
sentiment_dict_map = {'vader': vader_sentiment_dict,
                      'absolute liu hu': liu_hu_absolute_dict, 
                      'relative liu hu': liu_hu_relative_dict}
makeAdvancedPlotForAllSentimentDict(sentiment_dict_map)

In [ ]:
makeAdvancedPlotForAllSentimentDict({'vader': vader_sentiment_dict})

In [ ]:
sns.set(style="whitegrid", color_codes=True)
def makePlotForSentimentDict(sentiment_dict, y_min= -1.1, y_max = 1.1):
    plot_df = makeAdvancedDfFromDict('analysis', sentiment_dict)
    order = pd.Series(aggregateSentimentInformation(sentiment_dict)).sort_values().index # Order of columns in plot
    
    plt.ylim(y_min, y_max)
#     plot = sns.stripplot(x="country", y="sentiment", data = plot_df, order = order);
    plot = sns.boxplot(x="country", y="sentiment", data = plot_df, order=order);
    plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
    return plot

In [ ]:
makePlotForSentimentDict(vader_sentiment_dict)

In [ ]:
makePlotForSentimentDict(liu_hu_absolute_dict)

In [ ]:
makePlotForSentimentDict(liu_hu_relative_dict, -1.02, 1.02)

In [ ]:
makePlotForSentimentDict(liu_hu_ponderated_dict, -0.001, 0.08)

# Bonus question

In [ ]:
## Import receiver data
receiver_filename = os.path.join('hillary-clinton-emails','EmailReceivers.csv')
receiverdf = pd.read_csv(receiver_filename)
receiverdf.PersonId.max()

In [ ]:
df_email_initial.head(2)

In [ ]:
email_senders_map = {}
def makeEmailSendersMap(row):
    senderId = row.SenderPersonId
    if not np.isnan(senderId):
        email_senders_map[row.Id] = int(senderId)

In [ ]:
df_email_initial.apply(makeEmailSendersMap, axis=1)
email_senders_map[1]

In [ ]:
## Method to create the graph
import networkx as nx
sendReceiveGraph = nx.Graph()

def createGraphWithReceivers(row):
    email_id = row.EmailId
    receiver_id = row.PersonId
    if email_id and receiver_id and email_senders_map.get(email_id):
        sendReceiveGraph.add_edge(email_senders_map[email_id], receiver_id, {'email_id':email_id})

In [ ]:
receiverdf.apply(createGraphWithReceivers, axis=1)
print('finished')

In [ ]:
from operator import itemgetter
sorted(sendReceiveGraph.degree_iter(),key=itemgetter(1),reverse=True)

In [ ]:
d = nx.degree(sendReceiveGraph)
plt.figure(figsize=(20,15))
nx.draw(sendReceiveGraph, nodelist=d.keys(), node_size=[v * 20 for v in d.values()])

# plt.draw()
# plt.show()

In [ ]:
sendReceiveGraph.nodes()

In [ ]:
import community
partition = community.best_partition(sendReceiveGraph)

In [ ]:
size = float(len(set(partition.values())))
pos = nx.spring_layout(sendReceiveGraph)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(sendReceiveGraph, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))

plt.figure(figsize=(20,15))
nx.draw_networkx_edges(sendReceiveGraph,pos, alpha=0.5)
plt.show()

In [ ]:
set(partition.values())

In [ ]:
aa = {1: {'aa':2, 'bb': 3, 'cc': 1}, 2: {'bb': 5}}
aa[1]
import heapq
heapq.nlargest(3, aa[1], key=aa[1].get)

In [ ]:
partitionMailMap = {}  ## map between: {partition id -> { token -> occurence}}

for i in range(0,len(set(partition.values()))):
    partitionMailMap[i] = {}

def addMailWordsToPartition(row):
    senderId = row.SenderPersonId
    if not np.isnan(senderId):
        partitionNum = partition[senderId]
        tokenlist = tokenlistmap[row.name] #### ! IN CASE OF tokenlistmap ! 
        for token in tokenlist:
            if partitionMailMap[partitionNum].get(token):
                partitionMailMap[partitionNum][token] += 1
            else:
                partitionMailMap[partitionNum][token] = 0

In [ ]:
def getMostUsedWordsOfDictionary(token_dict):
    '''
    Input: Dictionary : {token -> occurence}
    Output: List[token_1, token_2, ..., token_20] with occurence(token_i) > occurence(token_j) for i > j
    '''
    return heapq.nlargest(20, token_dict, key=token_dict.get)

In [ ]:
print('We will have', len(set(partition.values())), 'communities.')
for partition_id in partitionMailMap:
    print('Community', partition_id,':')
    most_used_words_list = getMostUsedWordsOfDictionary(partitionMailMap[partition_id])
    print(most_used_words_list)
#     for word in most_used_words_list:
#         print(word)

In [ ]:
df_email_initial.RawText[0]
# stem_tokens[1]

In [ ]:
import networkx as nx


In [ ]:

G=nx.Graph()
# G.add_node(0)
# G.add_node(1)
G.add_edge('nice', 'hello', emailId=2)
G.add_edges_from([(2,3,{'color':'green'}), (3,4)])

In [ ]:

pos = nx.spring_layout(G)
edge_labels = nx.get_edge_attributes(G,'state')
nx.draw(G, pos)
nx.draw_networkx_edge_labels(G, pos, labels = edge_labels)

# Other

In [ ]:
plot_row_lists = []
for country in vader_sentiment_dict:
    for val in vader_sentiment_dict[country]:
        plot_row_lists.append({'country': country, 'sentiment': val})
plot_row_lists[:3]

In [ ]:
vader_result = aggregateSentimentInformation(vader_sentiment_dict)
pd.Series(vader_result).sort_values().index

In [ ]:
vader_result = aggregateSentimentInformation(vader_sentiment_dict)
pd.DataFrame.from_dict(vader_result, orient='columns', index='a')
results_series = pd.Series(vader_sentiment_dict, name='sentiments_val')
results_series.index.name = 'country'
plot_df = pd.DataFrame(results_series.reset_index())
plot_df.sort_values('sentiments_val', inplace=True)
plot_df.head(2)
plot_df = pd.DataFrame(plot_row_lists)

In [ ]:
sns.set(style="whitegrid", color_codes=True)
tips = sns.load_dataset("tips")

In [ ]:
plot = sns.stripplot(x="country", y="sentiment", data=plot_df);
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plot

In [ ]:
tips

In [ ]:
x = ['a', 'b', 'c']
b = ['a'] if 'aa' in x else ''
[a if a is 'a' else 'b' for a in x]

In [ ]:
print('hey')

In [ ]:
nltk.download('opinion_lexicon')
nltk.download('vader_lexicon')

In [ ]:
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import LinearSVC
import nltk.sentiment.util

naive_bayes = NaiveBayesClassifier.train
svm = SklearnClassifier(LinearSVC()).train
maxent = MaxentClassifier.train

nltk.sentiment.util.demo_vader_instance(df_email.ExtractedBodyText[13])

In [ ]:
nltk.sentiment.util.demo_liu_hu_lexicon(df_email.ExtractedBodyText[13], plot=True)

In [ ]:
def my_demo_liu_hu_lexicon(sentence, plot=False):
    """
    Basic example of sentiment classification using Liu and Hu opinion lexicon.
    This function simply counts the number of positive, negative and neutral words
    in the sentence and classifies it depending on which polarity is more represented.
    Words that do not appear in the lexicon are considered as neutral.
    :param sentence: a sentence whose polarity has to be classified.
    :param plot: if True, plot a visual representation of the sentence polarity.
    """
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank

    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    neut_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]

    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            print('positive: ',word)
            pos_words += 1
            y.append(1) # positive
        elif word in opinion_lexicon.negative():
            print('negative: ', word)
            neg_words += 1
            y.append(-1) # negative
        else:
            neut_words += 1
            y.append(0) # neutral
    print('Positive words: ', pos_words)
    print('Negative words: ', neg_words)
    print('Neutral words: ', neut_words)

    if pos_words > neg_words:
        print('Positive')
    elif pos_words < neg_words:
        print('Negative')
    elif pos_words == neg_words:
        print('Neutral')


In [ ]:
my_demo_liu_hu_lexicon(df_email.ExtractedBodyText[13])

In [ ]:
# df_email.ExtractedBodyText[13]

In [ ]:
pydic = {'Switzerland': {'CH':[0.2, 0.5, 1.5], 'Suisse': [1,2,3]}, 'Germany':{'GE': [2,2.1, 6, 8], 'Allemagne': [2]} }

In [ ]:
pydic.get('Switzerland').get('CH')

In [ ]:
pydic

In [ ]:
for a in pydic:
    print(pydic.get(a))

In [ ]:
if not pydic.get('Switzerlands'):
    pydic['Switzerlands'] = [2]
pydic

In [ ]:
testaaa = {'Switzerland': [0.2, 0.5, 1.5], 'Suisse': [1,2,3], 'Germany': [2,2.1, 6, 8], 'Allemagne': [2]}

In [ ]:
testaaa['aaa'] = [2]
testaaa

In [ ]:
update_sentiment_dict('key', testaaa, 25)
testaaa

In [ ]:
np.mean(testaaa['Germany'])

In [ ]:
(2 + 2.1 + 6 + 8)/4

# Old function:

In [ ]:
def analyzeSentimentInDfOLD(row):
    emailContent = str(row.ExtractedBodyText).lower()
#     if('syria' in emailContent):
#             if (row.ExtractedSubject):
#                 print("Subject: ",row.ExtractedSubject, "\n")
#             print(emailContent)
#             print('-----------------------------------------------------------------------------')
    foundCountryGlobal = False
    for country in countryNamesAbbrev:
        foundCountry = False
        for abrev in country:
            if not foundCountry and abrev in emailContent:
                if(not foundCountryGlobal):
                    print('------------------------------- EMAIL N0: ', row.Id, ' -------------------------------')
                    print('COUNTRIES:')
                foundCountry = True
                foundCountryGlobal = True
                print(country[0], ': ',abrev)
    if foundCountryGlobal:
        pass
        print('--------- EMAIL CONTENT ---------')

        print(emailContent)
#         if any(abrev in emailContent for abrev in country):
#             print(abrev)
#         print(emailContent)
#         if('9.11' in emailContent):
#             print(emailContent)
#             print('-----------------------------------------------------------------------------')
#             print('-----------------------------------------------------------------------------')

#         if any(abrev in emailContent for abrev in country):
#             print('hey')